In [1]:
import pandas as pd, numpy as np
import argparse
import pickle
import ast
import pandas as pd

In [3]:
with open("chexpertBinaryPNA_0_0_0_0_gpt-4o-2024-05-13_50.pkl", "rb") as f:
    raw_pickle = pickle.load(f)

In [4]:
for k,v in raw_pickle.items():
    print(len(v[2]))

50
50
50
50
50
50
50
50
50
50
50
50


TypeError: object of type 'int' has no len()

In [5]:
def pickle_to_res(path):
    with open(path, "rb") as f:
        raw_pickle = pickle.load(f)

    results = {}
    answer_prefix="Answer Choice "

    def extract_ans(ans_str, search_substring):
        # Split the string into lines
        lines = ans_str.split("\n")

        for line in lines:
            # Check if the line starts with the specified substring
            if line.startswith(search_substring):
                # If it does, add it to the list of extracted rows
                return line[len(search_substring) :].strip()
        return "ERROR"  # Answer not found

    for k, v in raw_pickle.items():
        if k != 'token_usage':  # Skip token_usage
            qns_idx = ast.literal_eval(k)
            for idx, qn_idx in enumerate(qns_idx):
                results[qn_idx] = extract_ans(
                    v[0], f"{answer_prefix}{idx+1}:"
                )  # We start with question 1
    
    return results

In [6]:
def res_to_vec(res):
    test_df = pd.read_csv('/home/joseph/datasets/chexpertchestxrays-u20210408/chexpert_binaryPNA_test_df_labels.csv', index_col=0)
    num_errors = 0
    labels, preds, race = [], [], []
    for i in test_df.itertuples():
        if (i.Index not in results) or (results[i.Index].startswith("ERROR")):
            num_errors += 1

            print(i.Index, f"answer not found")
            continue

        pred_text = results[i.Index]
        ground_truth = "B" if i.Pneumonia == 1 else "A"
        labels.append(ground_truth)
        preds.append(pred_text)
        race.append(i.binary_race)
        
    print(f"In total {num_errors} errors len = {len(labels)}")
    return labels,preds,race

In [7]:
def calculate_metrics(actual, predicted):
    TP = sum((a == 'B' and p == 'B') for a, p in zip(actual, predicted))
    TN = sum((a == 'A' and p == 'A') for a, p in zip(actual, predicted))
    FP = sum((a == 'A' and p == 'B') for a, p in zip(actual, predicted))
    FN = sum((a == 'B' and p == 'A') for a, p in zip(actual, predicted))
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    # Calculate F1 score
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0
    
    accuracy = (TP + TN) / len(actual)
    tpr = TP / (TP + FN) if (TP + FN) > 0 else 0
    tnr = TN / (TN + FP) if (TN + FP) > 0 else 0
    
    return accuracy, tpr, tnr, f1_score



def filter_by_race(actual, predicted, race, race_value):
    filtered_actual = [a for a, r in zip(actual, race) if r == race_value]
    filtered_predicted = [p for p, r in zip(predicted, race) if r == race_value]
    return filtered_actual, filtered_predicted

In [8]:
results = pickle_to_res("chexpertBinaryPNA_0_0_0_0_gpt-4o-2024-05-13_50.pkl")
labels,preds,race = res_to_vec(results)
accuracy, tpr, tnr, fscore = calculate_metrics(labels, preds)

In total 0 errors len = 600


In [10]:
accuracy, tpr, tnr, fscore

(0.45166666666666666, 0.74, 0.16333333333333333, 0.574385510996119)

In [11]:
# Filter for race white
actual_white, predicted_white = filter_by_race(labels, preds, race, 'White')
accuracy_white, tpr_white, tnr_white, fscore_white = calculate_metrics(actual_white, predicted_white)

# Filter for race black
actual_black, predicted_black = filter_by_race(labels, preds, race, 'Black')
accuracy_black, tpr_black, tnr_black, fscore_black = calculate_metrics(actual_black, predicted_black)

In [13]:
accuracy_black,accuracy_white

(0.45, 0.4533333333333333)